In [3]:
# Exploration: siblings are weird

c = 0
for title, url, sib in zip(raw['title'], raw['url'], raw['siblings']):
    if type(sib) == list:
        print(title, url, sib)

    c += 1
    if c == 10:
        break

NameError: name 'raw' is not defined

In [ ]:
# Web entity detection in Google's Vision AI data
vision.head()

c = 0
print(instances.keys())
print()
for instances in vision['webDetection']:
    if 'partialMatchingImages' in instances:
        print(instances['partialMatchingImages'])
    print()
    #print(instances['webEntities'])

    c += 1
    if c == 20:
        break